In [1]:
import requests
from bs4 import BeautifulSoup



In [2]:
def normalize_names(country_name):
    res = country_name
    res = res.lower()
    res = res.replace(',', '')
    res = res.replace('-', '')
    res = res.replace('republic of the', '')
    res = res.replace('democratic', '')
    res = res.replace('federated states of', '')
    res = res.replace('cabo', 'cape')
    res = res.strip()
    res = res.title()
    res = res.replace('And', 'and')
    res = res.split(' (')[0]
    return res


In [3]:
def get_continent_pop_prospect():
    url = 'https://www.countries-ofthe-world.com/continents-of-the-world.html'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                             ' Chrome/87.0.4280.88 Safari/537.36'}

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')

    pop_prospect = {}

    for row in soup.find_all('table')[1].tbody.find_all('tr'):
        tds = row.find_all('td')
        pop_prospect[tds[1].text] = tds[-1].text
    return pop_prospect

In [4]:
def get_country_domains():
    url = 'https://www.countries-ofthe-world.com/TLD-list.html'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                             ' Chrome/87.0.4280.88 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')
    domains = {}
    for table in soup.find_all('table'):
        for row in table.tbody.find_all('tr'):
            tds = row.find_all('td')
            if len(tds) > 1 and not tds[1].has_attr('class'):
                domains[normalize_names(tds[1].text)] = tds[0].text

    return domains


In [5]:
def get_country_timezones():
    url = 'https://www.countries-ofthe-world.com/world-time-zones.html'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/87.0.4280.88 Safari/537.36'}

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')

    timezones = {}
    for row in soup.tbody.find_all('tr'):
        tds = row.find_all('td')

        if not tds[0].has_attr('class'):
            timezones[normalize_names(tds[0].text)] = tds[1].text
        elif tds[0]['class'][0] == 'mlt':
            url2 = 'https://www.countries-ofthe-world.com/'+tds[0].a['href']
            response2 = requests.get(url2, headers=headers)
            soup2 = BeautifulSoup(response2.content, 'lxml')
            nested_timezones = set()
            for i in soup2.table.tbody.find_all('tr'):
                if len(i.find_all('td')) >= 2:
                    nested_timezones.add(i.find_all('td')[-2].text)
            timezones[tds[0].span.text] = list(nested_timezones)
        elif row.has_attr('class') and row['class'][0] == 'green':
            if tds[0].text[:-3].split(',')[0] in timezones:
                if type(timezones[tds[0].text[:-3].split(',')[0]]) == str:
                    first_item = timezones[tds[0].text[:-3].split(',')[0]]
                    timezones[tds[0].text[:-3].split(',')[0]] = {first_item}
                timezones[tds[0].text[:-3].split(',')[0]].add(tds[1].text)
            else:
                timezones[tds[0].text[:-3].split(',')[0]] = {tds[1].text}
    for item in timezones.keys():
        if type(timezones[item]) == set:
            timezones[item] = list(timezones[item])
    return timezones


In [6]:
def get_country_codes():
    url = 'https://www.countries-ofthe-world.com/list-of-country-calling-codes.html'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/87.0.4280.88 Safari/537.36'}

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')

    calling_codes = {}
    for row in soup.tbody.find_all('tr'):
        if not row.find_all('td')[1].has_attr('class'):
            country_name = normalize_names(row.find_all('td')[1].text)
            code = row.find_all('td')[0].text
            calling_codes[country_name] = code
    del calling_codes['Australian External Territories']
    return calling_codes


In [7]:
pop_prospect_data = get_continent_pop_prospect()
code_data = get_country_codes()
timezones_data = get_country_timezones()
domains_data = get_country_domains()

